In [1]:
%load_ext rpy2.ipython

In [2]:
%%R
install.packages("PMCMRplus", dependencies=TRUE, quiet=TRUE, repos="https://cran.ma.imperial.ac.uk/")
library(PMCMRplus)

R[write to console]: also installing the dependencies ‘classInt’, ‘questionr’, ‘MatrixModels’, ‘conquer’, ‘openxlsx’, ‘RcppEigen’, ‘klaR’, ‘survival’, ‘htmlTable’, ‘cubature’, ‘pbkrtest’, ‘quantreg’, ‘rio’, ‘lme4’, ‘TH.data’, ‘agricolae’, ‘coin’, ‘Hmisc’, ‘km.ci’, ‘metafor’, ‘np’, ‘car’, ‘multcomp’, ‘NSM3’


R[write to console]: Updating HTML index of packages in '.Library'

R[write to console]: Making 'packages.html' ...
R[write to console]:  done



In [3]:
import sys
import numpy as np
import pandas as pd
import seaborn as sns
from scipy import stats
from typing import List

import matplotlib.pyplot as plt

%matplotlib inline

In [8]:
import constants

In [10]:
import cancer_package.constants

ModuleNotFoundError: No module named 'transformers'

In [6]:
#import cancer_package.constants

INTERPRETABLE_DIR = "interpretable_results/"
PROTEIN_SET = "2nd_set_proteins/"

USE_ENERGY_PROTEINS = False
NON_NAN_THRESH = .7
REPLACE_BY_NAN = False
N_FOLDS = 10
ENERGY_PROTEINS = ['CAT', 'FBP1', 'FBP2', 'GCLC', 'GCLM', 'GGT1', 'GGT6', 'GSR',
       'GSS', 'GSTA1', 'GSTA2', 'GSTK1', 'GSTM1', 'GSTM2', 'GSTM3',
       'GSTO1', 'GSTP1', 'GSTT1', 'GSTZ1', 'MGST1', 'MGST2', 'MGST3',
       'SDHA', 'SDHB', 'SOD1', 'SOD2', 'SOD3', 'SRC']

ModuleNotFoundError: No module named 'transformers'

In [ ]:
data = pd.read_csv( constants.DATA_DIRECTORY + 'cancer_and_ld_patients.csv')
protein_group = pd.read_csv(constants.DATA_DIRECTORY + 'protein_group.csv')

# checking we have data
print(f"original dataset shape: {data.shape}")

data_prep = BasicPreprocessing(data, constants.NA_VALUE)
data_prep.non_nan_share_hist(bins=30)
data_prep.rm_execess_nans(NON_NAN_THRESH, by_group=False)
data_prep.organise_proteins(protein_group)

if REPLACE_BY_NAN:
  data_prep.replace_nans()
  print("replaced NaNs")

if not USE_ENERGY_PROTEINS:
  data_prep.rm_energy_proteins(ENERGY_PROTEINS)
  print("removed energy proteins")

data = data_prep.data
proteins = data_prep.proteins

print(f"preprocessed dataset shape: {data.shape}")
print(data.category.value_counts())

In [ ]:
"""
fig, _ = plt.subplots(1, 1, figsize=(20, 20)) 
sns.heatmap(data[proteins].corr())

fig, _ = plt.subplots(1, 1, figsize=(20, 20)) 
sns.heatmap(data[proteins].corr("spearman"))
"""

In [ ]:
"""
for my_protein in proteins:
  sns.histplot(
      #data=pd.concat([np.log(data[my_protein]), data["category"]], axis=1),
      data=data[[my_protein, "category"]],
      x=my_protein,
      hue="category",
      element="step"
  ) 
  plt.show()
"""

In [ ]:
#https://www.statsmodels.org/dev/generated/statsmodels.stats.multitest.multipletests.html

from scipy.stats import kruskal
kruskal_p_vals = {my_protein:
  kruskal(
      *[group[my_protein].values 
        for _, group in data[~data.category.str.contains("LD")].groupby("category")]
  ).pvalue
  for my_protein in proteins
}

n_h0_rejections = sum([val < .05 for val in kruskal_p_vals.values()])
print(f"number of rejections of the null according without correction {n_h0_rejections}")

In [ ]:
from statsmodels.stats.multitest import multipletests

reject_h0_corrected, pvals_corrected, _, _ = multipletests(
    list(kruskal_p_vals.values()),
    method="holm", #"fdr_bh",
    alpha=0.05
)

proteins_kw_test = pd.DataFrame(
    {
        "protein": kruskal_p_vals.keys(),
        "p_val_corrected": pvals_corrected,
        "reject_h0": reject_h0_corrected
    }
)

kw_significant_proteins = proteins_kw_test[proteins_kw_test.reject_h0].protein

print(f"we reject H_0 for {proteins_kw_test.reject_h0.sum()}")
display(proteins_kw_test)

In [ ]:
cancer_ctl_data = data[~data.category.str.contains("LD")]
cancer_types = ['LC', 'BC', 'CCA', 'CRC', 'EC', 'GC']
%R -i cancer_ctl_data
%R -i kw_significant_proteins
%R -i cancer_types

In [ ]:
%%R

cancer_types = unlist(cancer_types)
cancer_ctl_data$category <- factor(
    cancer_ctl_data$category,
    ordered = TRUE,
    levels=c('CTL', cancer_types)
)


all_statistics = c()
all_p_values = c()
for (col in kw_significant_proteins) {
    hyp_test = kwManyOneDunnTest(
        cancer_ctl_data[[col]], 
        cancer_ctl_data$category,
        alternative = "two.sided",
        p.adjust.method = "holm"
    )
    print(paste("-----------------------------   ", col, "  ---------------------------"))
    summary(hyp_test)
    hyp_test_statistic = hyp_test$statistic
    colnames(hyp_test_statistic) = col
    all_statistics = cbind(all_statistics, hyp_test_statistic)
    
    hyp_test_p_values = hyp_test$p.value
    colnames(hyp_test_p_values) = col
    all_p_values = cbind(all_p_values, hyp_test_p_values)
}

In [ ]:
%R -o all_statistics
%R -o all_p_values

In [ ]:
all_statistics = pd.DataFrame.from_records( all_statistics, columns=list(kw_significant_proteins))
all_statistics.columns = pd.MultiIndex.from_product([all_statistics.columns, ["stat"]])

all_p_values = pd.DataFrame.from_records( all_p_values, columns=list(kw_significant_proteins))
all_p_values.columns = pd.MultiIndex.from_product([all_p_values.columns, ["p-value"]])

In [ ]:
post_hoc = pd.concat([all_p_values, all_statistics], axis=1)
post_hoc = post_hoc.sort_index(axis=1,level=[0,1],ascending=[True,False])
post_hoc["category"] = cancer_types
post_hoc.set_index("category", inplace=True)
post_hoc

In [ ]:
from matplotlib import colors
import seaborn as sns

def b_g(s):
    cm=sns.light_palette((260, 75, 60), input="husl", as_cmap=True, reverse=True)
    norm = colors.Normalize(0,.025)
    normed = norm(abs(s.values))
    c = [colors.rgb2hex(x) for x in plt.cm.get_cmap(cm)(normed)]
    return ['background-color: %s' % color for color in c]

post_hoc.style.apply(b_g, subset=all_p_values.columns)